In [1]:
import tensorflow as tf
from scipy import stats
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python import debug as tf_debug

In [3]:
#Generate an underlying permutation matrix.
# Global parameters
#np.random.seed(12)
#Permutation and weight matrices are KxK
sess = tf.Session()
#sess = tf_debug.LocalCLIDebugWrapperSession(sess)
#sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

K = 5
num_samples = 10
#Don't have an annealing schedule
tau = tf.constant(0.75)
# Sample a true permutation (in=col, out=row)
#perm_true = np.random.permutation(np.arange(K))
#P_true = np.zeros((K, K))
#P_true[np.arange(K), perm_true] = 1
#P_true = np.float32(P_true)
P_true = np.eye(K)
print(P_true)
#Generate an underlying weight matrix.
W_true = np.float32(np.random.randn(K,K))
print(W_true)
#Generate ficticious data from the permutation and weight matrix.
#x_i = PWP^T*a_i+epsilon, i.e. x_i~N(PWP^T*a_i,sigma_true) for epsilon~N(0,sigma_true)
noise_scale = 0.1
RChol = np.array(np.random.rand(K,K)*noise_scale,dtype='float32')
sigma_true = RChol.dot(RChol.T)
#Now generate the a_i.
a = np.array(np.random.rand(num_samples,K,1),dtype='float32')
#Now the data x that will be observed.
PWP = P_true.dot(W_true).dot(P_true.T)
x = np.matmul(PWP,a)
x = tf.constant(np.float32(x))
a = tf.constant(np.float32(a))
#Could be a parameter to be learned, but set to constant for now.
sigmaChol = np.array(np.random.rand(K,K)*noise_scale,dtype='float32')

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
[[ 0.10077184  1.6843127   0.4835811  -1.09100914  0.13907139]
 [-0.83964658  0.80931187 -1.2240566   0.80600601 -1.37996602]
 [ 0.71245444 -0.23467234  0.44525203 -0.87935728  0.42449108]
 [ 0.24505809  0.16287078  1.39144504  0.97881103  0.3398042 ]
 [-1.22279549 -1.71691489 -0.85684156  0.59503055  1.17687225]]


In [4]:
#Variational model needs to return both a permutation sample P, and a weight sample W
#Weight sample W for now will be independent normals for each Wij.
#P is according to our rounding method.

mu_W = tf.Variable(np.array(np.ones([K,K]),dtype='float32'))
#Initialise small noise leads to better training results in my experience.
sigma_W = tf.Variable(np.array(np.ones([K,K])*0.5,dtype='float32'))
#Comp stability hacks. Change to tau?
sigma_W_pos = sigma_W**2+10E-5
tf.assert_positive(sigma_W_pos)
#Define a tensorflow matrix of independence normal random variables for the variational posterior.
q_W = tf.contrib.distributions.Normal(loc=mu_W, scale=sigma_W_pos)

#Now want to define the variational posterior for the permutation.
mu_P = tf.Variable(np.array(np.random.rand(K,K),dtype='float32'))
mu_P = mu_P**2+10E-5
tf.assert_positive(mu_P)


#Scott's Sinkhorn in logspace
def sinkhorn_logspace(logP, niters=3):
    for _ in range(niters):
        # Normalize columns and take the log again
        logP = logP - tf.reduce_logsumexp(logP, axis=0,keep_dims=True)
        # Normalize rows and take the log again
        logP = logP - tf.reduce_logsumexp(logP, axis=1,keep_dims=True)
    return logP
#Remember to exp() again to get the matrix out of log-space.    
mu_P_sinkhorned = tf.exp(sinkhorn_logspace(tf.log(mu_P),niters=5))
#vectorize the matrix
mu_P_vec = tf.reshape(mu_P_sinkhorned,shape=[-1])
#eta_P is the diagonal of a matrix that we draw permutation samples from.
eta_P = tf.Variable(np.array(np.random.rand(K**2,1),dtype='float32'))
#The bounds that seem to be suggested in Gonzalo's code.
lb = 0.001
ub = 0.01
#Enforcing the bounds.
eta_P_constrained = tf.reshape(lb+(ub-lb)*tf.nn.sigmoid(eta_P),shape=[-1])
tf.assert_positive(eta_P_constrained)
#Creating another tensorflow normal distribution according to the model discussed on
#slack.
q_P = tf.contrib.distributions.Normal(loc=mu_P_vec,scale = eta_P_constrained)
def variational_sample():
    #Works by default for the tensorflow distribution object
    W_sample = q_W.sample()
    #Now need a permutation sample. Start by reshaping hte draw from q_P into a KxK matrix
    phi = tf.reshape(q_P.sample(),shape=[K,K])
    #Now have to round using Hungarian. But there is no tensorflow implementation. So we evaluate
    #using sess.run, and run the standard numpy. It's probably better to use "with Session as sess" etc and indent
    #for all this, but that makes it more difficult to use the jupyter notebook cells.
    #So now just generate the sample as in Gonzalo's notebook.
    row, col = optimize.linear_sum_assignment(-sess.run(phi))
    rounded_phi = np.zeros((K, K))
    rounded_phi[row, col] = 1.0
    rounded_phi =  tf.constant(np.float32(rounded_phi))
    P_sample = tau*phi+(1-tau)*rounded_phi
    #Remember returns the sample for both parts of the variational posterior.
    #The following two lines can be used to check inference on each parameter individually.
    #P = tf.constant(P_true)
    #W = tf.constant(W_true)
    return [W_sample,P_sample]

#This gives a single sample approximation for the expectation of the log prob of q(z|x).
def variational_log_prob(W_sample):
    #Log probability of the sample of W
    var_log_prob = tf.reduce_sum(q_W.log_prob(W_sample))
    #Log probability of the sample of P, just the entropy of the normal from the sampling process. Handy.
    entropy_phi = q_P.entropy()
    return var_log_prob

In [5]:
#Need to create these now to allow for the Hungarian work-around to work. Drawing samples requires the parameters to
#be defined because Hungarian requires actual values. But we have to initialise again later to define the optimization
#related parameters.
sess.run(tf.global_variables_initializer())
W_sample,P_sample = variational_sample()
#Need to be careful with training algorithm and learning rate. Despite precautions, seem to veer into NaN territory
#at times.
#optimizer = tf.train.AdagradOptimizer(learning_rate = 1.0)
optimizer = tf.train.AdamOptimizer(0.01)
#optimizer = tf.train.FtrlOptimizer(learning_rate = 0.5)
#optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.002)
#optimizer = tf.train.MomentumOptimizer(learning_rate = 0.001,momentum = 0.01)
#optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=0.0001)
#optimizer = tf.train.RMSPropOptimizer(0.00005)


In [6]:
#logpdf for multivariate normal since tensorflows mvn distribution is currently limited.
def tf_mvn_logpdf(X,Mu,XChol):
    ''' Use this version when X is a vector '''
    Lambda = tf.matrix_inverse(tf.matmul(XChol,tf.transpose(XChol)))
    #Lambda = tf.matmul(XChol,tf.transpose(XChol))
    XMu    = X-Mu
    return_value= (-0.5 * tf.matmul(XMu, tf.matmul(Lambda,tf.transpose(XMu)))
                  +0.5 * tf.log(tf.matrix_determinant(Lambda))
                  -0.5 * np.log(2*np.pi) * int(X.shape[1]))       
    return return_value

In [7]:
#We assume we know both x and a, with both P and W have uniform prior distributions
def generative_log_prob(x,a,P,W,sigmaChol):
    #Here x and a are data, but P and W are tensorflow variable samples from the variational posterior
    #In accordance with x_i~N(PWP^T*a_i,sigma_true)
    A = tf.matmul(P,tf.matmul(W,tf.transpose(P)))
    #Below just encodes the prior over the x's. Simply repeatedly call tf_mvn_logpdf for each sample. Can do this
    #quicker for a tf_mvn_logpdf that acts on a matrix of X's rather than a vector, but I didn't have time to reimpl
    #ement int.
    indices = tf.constant(np.arange(num_samples,dtype='int32'))
    def mvn_calls(index):
        return tf_mvn_logpdf(tf.transpose(x[index]),tf.transpose(tf.matmul(A,a[index])),sigmaChol)
    gen_log_prob = tf.reduce_sum(tf.map_fn(mvn_calls,indices,dtype='float32'))    
    #Prior on normals to be standard uniform.
    W_check = tf.contrib.distributions.Normal(loc=np.array(np.zeros([K,K]),dtype='float32'),
                                              scale = np.array(np.ones([K,K]),dtype='float32'))
    gen_log_prob = gen_log_prob + tf.reduce_sum(W_check.log_prob(W))
    return gen_log_prob

In [9]:
#Single sample approximation of the ELBO.
ELBO = generative_log_prob(x,a,P_sample,W_sample,sigmaChol)-variational_log_prob(W_sample)
print(W_true)
print(P_true)

[[ 0.10077184  1.6843127   0.4835811  -1.09100914  0.13907139]
 [-0.83964658  0.80931187 -1.2240566   0.80600601 -1.37996602]
 [ 0.71245444 -0.23467234  0.44525203 -0.87935728  0.42449108]
 [ 0.24505809  0.16287078  1.39144504  0.97881103  0.3398042 ]
 [-1.22279549 -1.71691489 -0.85684156  0.59503055  1.17687225]]
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


In [12]:
# gradients, variables = zip(*optimizer.compute_gradients(-ELBO))
# gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
# train_fn = optimizer.apply_gradients(zip(gradients, variables))
train_fn = optimizer.minimize(-ELBO)
train_steps=50000
grad_checker = tf.gradients(-ELBO,tf.trainable_variables())
#check_op = tf.add_check_numerics_ops()
#Need to run initializer twice to allow for the optimizer to be instantiated. But need to run it earlier in
#order to get around the lack of a Tensorflow hungarian algorithm.
sess.run(tf.global_variables_initializer())
#####
ascent_tracker = np.array([np.nan]*train_steps)
for step in range(train_steps):
    grad_check_print = sess.run(grad_checker)
    mu_W_print,sigma_W_print,mu_P_print,eta_P_print = sess.run([mu_W,sigma_W,mu_P,eta_P])
    sess.run([train_fn])
    ascent_tracker[step]=sess.run(ELBO)
    if (step%100==0):
        print(step, sess.run([ELBO]))
        print(sess.run([tf.round(P_sample)]))
        if np.isnan(sess.run(ELBO)):
            break
        #print(sess.run(grad_checker))
        



0 [-25541.188]
[array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]], dtype=float32)]
100 [-4448.3242]
[array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)]
200 [-2277.355]
[array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]], dtype=float32)]
300 [-1263.4819]
[array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]], dtype=float32)]
400 [-753.57898]
[array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  

KeyboardInterrupt: 